In [1]:
import pandas as pd 
import seaborn as sns 
import sqlite3
import numpy as np
from plotly import express as px
import ipywidgets as widgets

with sqlite3.connect("../tickets.db") as conn: 
    cmd = \
        """
        SELECT g.*, e.price, e.extractTime, e.ticket_count FROM extraction e
        LEFT JOIN games g on e.id = g.id
        WHERE g.season = 2022
        """
    df = pd.read_sql_query(cmd, conn)
df['time'], df['extractTime'] = pd.to_datetime(df['time']), pd.to_datetime(df['extractTime'])
df['days_to_game'] = (df['time'] - df['extractTime'])
df['days_to_game'] = df['days_to_game'].apply(lambda X: X.days)
df = df.dropna()
def diff_from_max(x):
    initial = x.to_numpy()[0]
    return (initial - x)/x
df["diff_initial"] = df.groupby("unique_name")["price"].transform(diff_from_max)
df = df[df['extractTime'] > '2022-03-01']
dodgers = df[df['homeTeam'] == 'Los Angeles Dodgers']
giants = df[df['homeTeam'] == 'San Francisco Giants']
angels = df[df['homeTeam'] == 'Los Angeles Angels']

# Dodgers

In [2]:
from ipywidgets import interact
team_games = {team: df[df["homeTeam"] == team]["unique_name"].unique() for team in df["homeTeam"].sort_values().unique()}

teamW = widgets.Dropdown(options = team_games.keys(), value = "Los Angeles Dodgers")
gameW = widgets.Dropdown()

@interact(team = teamW, game = gameW)
def print_city(team, game):
    gameW.options = team_games[team]
    ids = df.loc[df["unique_name"] == game, "id"].unique()[0]
    gm1 = df[df["id"] == ids].sort_values(by="extractTime", ascending=True)
    fig = sns.lineplot(data = gm1, x = "extractTime", y = "price") # Here is the trick, i.e. update cityW.options based on country, namely countryW.value.
    sns.set(rc={"figure.figsize":(15, 6)})
    fig.tick_params('x', rotation=45)

interactive(children=(Dropdown(description='team', index=13, options=('Arizona Diamondbacks', 'Atlanta Braves'…